In [26]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import json

In [31]:
API_KEY = "RGAPI-1d7ef4e4-7b8a-4d8e-98c0-e01e8a6c896a"
GAME_NAME = "X8Scorpio8X"
TAG_LINE = "NA1"  # do not include "#"
REGION = "americas"

def get_puuid_from_riot_id(game_name, tag_line, api_key, region):
    # URL-encode the game name and tag line
    game_name_encoded = quote(game_name)
    tag_line_encoded = quote(tag_line)
    
    url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name_encoded}/{tag_line_encoded}"
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data["puuid"]
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
puuid = get_puuid_from_riot_id(GAME_NAME, TAG_LINE, API_KEY, REGION)
if puuid:
    print("PUUID:", puuid)


PUUID: vrfSU4Nh3GYoxHi2hLXg_yOczDqdE8WulZzWDLl2RZu6wtmH-e8i629HTEVbu1Uej1CpJZ3JIfXu3w


In [33]:
PUUID = puuid
REGION = "americas"  # match routing region

def get_match_ids(puuid, api_key, region, count=20):
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {
        "start": 0,
        "count": count  # number of matches to retrieve (max 100 per call)
    }
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # list of match IDs
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
match_ids = get_match_ids(PUUID, API_KEY, REGION)
if match_ids:
    print("Recent Match IDs:", match_ids)


Recent Match IDs: ['NA1_5301958274', 'NA1_5301679760', 'NA1_5301539074', 'NA1_5300777438', 'NA1_5300767414', 'NA1_5299827201', 'NA1_5299689498', 'NA1_5299540428', 'NA1_5299303966', 'NA1_5299258676', 'NA1_5299227403', 'NA1_5299065618', 'NA1_5299054370', 'NA1_5299036599', 'NA1_5299017701', 'NA1_5298812710', 'NA1_5298785933', 'NA1_5298162104', 'NA1_5298134070', 'NA1_5298096012']


In [34]:
def parse_match_data(match_data):
    if match_data['info']['queueId'] != 420:  # Only solo/duo ranked games
        return []

    teams = match_data['info']['teams']
    participants = match_data['info']['participants']

    # Split participants by teamId
    team1_id = teams[0]['teamId']
    team2_id = teams[1]['teamId']
    team1_players = [p for p in participants if p['teamId'] == team1_id]
    team2_players = [p for p in participants if p['teamId'] == team2_id]

    def team_stats(team, players, other_players):
        def get_ping_sum(key):
            return sum(p.get(key, 0) for p in players)

        # Map team positions to column names
        position_map = {
            'TOP': 'top',
            'JUNGLE': 'jung',
            'MIDDLE': 'mid',
            'BOTTOM': 'bot',
            'UTILITY': 'sup'
        }

        
        # Champion name and items per role
        role_data = {}
        for p in players:
            pos = p.get('teamPosition')
            if pos not in position_map:
                continue
            role = position_map[pos]
            champ = p.get('championName')
            items = [p.get(f'item{i}', 0) for i in range(7)]
            role_data[role] = champ
            role_data[f'{role}Items'] = items


        return {
            'matchId': match_data['metadata']['matchId'],
            'teamId': team['teamId'],
            'win': team['win'],
            'baronKills': team['objectives']['baron']['kills'],
            'dragonKills': team['objectives']['dragon']['kills'],
            'riftHeraldKills': team['objectives']['riftHerald']['kills'],
            'totalGold': sum(p['goldEarned'] for p in players),
            'goldSpent': sum(p['goldSpent'] for p in players),
            'totalKills': sum(p['kills'] for p in players),
            'totalDeaths': sum(p['deaths'] for p in players),
            'totalTurretsTaken': sum(p.get('challenges', {}).get('turretTakedowns', 0) for p in players),
            'totalInhibitorsTaken': sum(p['inhibitorKills'] for p in players),
            'averageChampionExp': sum(p['champExperience'] for p in players) / len(players),
            'wardsPlaced': sum(p['wardsPlaced'] for p in players),
            'goldDiff': sum(p['goldEarned'] for p in players) - sum(p['goldEarned'] for p in other_players),
            'allInPings': get_ping_sum('allInPings'),
            'assistMePings': get_ping_sum('assistMePings'),
            'commandPings': get_ping_sum('commandPings'),
            'enemyMissingPings': get_ping_sum('enemyMissingPings'),
            'enemyVisionPings': get_ping_sum('enemyVisionPings'),
            'holdPings': get_ping_sum('holdPings'),
            'getBackPings': get_ping_sum('getBackPings'),
            'totalPings': (
                get_ping_sum('allInPings') +
                get_ping_sum('assistMePings') +
                get_ping_sum('commandPings') +
                get_ping_sum('enemyMissingPings') +
                get_ping_sum('enemyVisionPings') +
                get_ping_sum('holdPings') +
                get_ping_sum('getBackPings')
            ),
            'gameDurationMinutes': match_data['info']['gameDuration'] / 60,
            **role_data
        }

    return [
        team_stats(teams[0], team1_players, team2_players),
        team_stats(teams[1], team2_players, team1_players)
    ]


def collect_match_stats(match_ids, api_key):
    all_rows = []
    for i, match_id in enumerate(match_ids):
        url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
        headers = {"X-Riot-Token": api_key}
        
        while True:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                match_data = response.json()
                rows = parse_match_data(match_data)
                all_rows.extend(rows)
                break
            elif response.status_code == 429:
                print("Rate limit exceeded. Sleeping for 60 seconds...")
                time.sleep(60)
            else:
                print(f"Failed to retrieve match {match_id}: {response.status_code}")
                break

        time.sleep(1.3)

    return pd.DataFrame(all_rows)



In [35]:
#TESTER SESSION

names_taglines = {"X8Scorpio8X":"NA1"}
puuids = []
region = "americas"

# getting puuids of silver players
for GAME_NAME in names_taglines:
    puuids.append(get_puuid_from_riot_id(GAME_NAME, names_taglines[GAME_NAME], API_KEY, region))

# getting match_ids from players' recent games
match_ids = []
for puuid in puuids:
    match_ids.append(get_match_ids(puuid, API_KEY, region, count=20))

match_ids = np.array(match_ids).flatten().tolist()



# collecting match stats
scorpioData = collect_match_stats(match_ids, API_KEY)

In [37]:
scorpioData.to_csv("X8Scorpio8X_Data")
scorpioData.head()

,matchId,teamId,win,baronKills,dragonKills,riftHeraldKills,totalGold,goldSpent,totalKills,totalDeaths,...,top,topItems,jung,jungItems,mid,midItems,bot,botItems,sup,supItems
0,NA1_5301958274,100,False,0,0,1,56995,54100,24,40,...,Garen,"[6631, 3046, 6675, 3044, 3067, 3006, 3340]",Nocturne,"[6692, 6696, 3111, 3814, 0, 0, 3340]",Samira,"[3072, 3111, 6676, 3156, 3031, 1018, 3363]",Jinx,"[1055, 3006, 3032, 3085, 0, 0, 3340]",Teemo,"[3020, 3118, 3871, 6653, 3916, 1028, 3364]"
1,NA1_5301958274,200,True,1,5,0,66540,59250,38,24,...,Darius,"[6631, 2019, 3053, 3174, 3742, 1036, 3340]",Nunu,"[3075, 6653, 3174, 3068, 0, 1028, 3340]",Hwei,"[1056, 4646, 2031, 4645, 3175, 3118, 3364]",Tristana,"[1055, 3032, 3031, 3172, 6675, 3036, 3340]",Rell,"[3876, 3190, 3174, 2055, 3075, 3084, 3364]"
2,NA1_5301679760,100,False,0,1,0,43894,39575,24,36,...,Naafiri,"[3047, 3070, 1036, 6692, 3123, 3133, 3363]",Nunu,"[1031, 6653, 3068, 3047, 1028, 0, 3340]",Garen,"[1054, 6631, 3009, 3046, 3044, 0, 3340]",Ashe,"[1055, 2031, 6672, 3006, 1018, 1057, 3340]",Mel,"[3871, 2503, 3020, 6653, 1026, 0, 3364]"
3,NA1_5301679760,200,True,0,2,1,53923,46650,36,24,...,Aatrox,"[1054, 2031, 3161, 3047, 2021, 3133, 3340]",Warwick,"[3111, 6672, 6631, 3067, 0, 0, 3340]",Malzahar,"[2503, 3118, 3175, 6653, 0, 0, 3340]",Twitch,"[1055, 3153, 3170, 3033, 3085, 1038, 3340]",Brand,"[6653, 3020, 3116, 0, 3871, 0, 3340]"
4,NA1_5301539074,100,False,0,2,0,52886,48450,28,32,...,Kayle,"[1026, 3115, 3153, 3006, 3916, 2031, 3340]",Shaco,"[6653, 2503, 4645, 0, 3158, 0, 3364]",Yone,"[3031, 3006, 2003, 0, 3153, 0, 3364]",Aphelios,"[3031, 3172, 6676, 2031, 3123, 1018, 3363]",Seraphine,"[3871, 3114, 2022, 3137, 2503, 3171, 3340]"


Collecting data from people who are in silver rank

In [24]:
names_taglines = {"Paro":"0820", "MicDonuld":"NA1", "prohanp":"NA1", "eGirl Trainer": "queer", "iSejuani":"NA1", "Fartgas12":"NA1", "Guymandudemanguy": "Ocean", "7hunder3":"Lucky", "WHITEsmoke96":"NA1", "grinder enjoyer":"nocap", 
                  "Atomic":"8541", "LucQie":"NA1", "Konlen":"NA1", "Hide Inside Lux":"NA1", "Valdivian":"NA1", "Gray Goo":"NA1"}
puuids = []
region = "americas"

# getting puuids of silver players
for GAME_NAME in names_taglines:
    puuids.append(get_puuid_from_riot_id(GAME_NAME, names_taglines[GAME_NAME], API_KEY, region))

# getting match_ids from players' recent games
match_ids = []
for puuid in puuids:
    match_ids.append(get_match_ids(puuid, API_KEY, region, count=20))

match_ids = np.array(match_ids).flatten().tolist()

In [25]:
# collecting match stats
silver_data = collect_match_stats(match_ids, API_KEY)

Rate limit exceeded. Sleeping for 60 seconds...


KeyboardInterrupt: 

In [ ]:
silver_data.to_csv("silver_data")
silver_data

,matchId,teamId,win,baronKills,dragonKills,riftHeraldKills,totalGold,goldSpent,totalKills,totalDeaths,...,goldDiff,allInPings,assistMePings,commandPings,enemyMissingPings,enemyVisionPings,holdPings,getBackPings,totalPings,gameDurationMinutes
0,NA1_5299855891,100,False,0,3,0,67323,62633,42,58,...,-10274,13,33,25,73,14,0,19,177,31.850000
1,NA1_5299855891,200,True,1,2,1,77597,68075,58,42,...,10274,0,15,42,10,9,0,4,80,31.850000
2,NA1_5299818587,100,True,1,2,0,64931,56800,33,24,...,11862,8,13,18,34,2,0,5,80,29.933333
3,NA1_5299818587,200,False,0,2,1,53069,48575,24,33,...,-11862,1,5,10,21,11,0,33,81,29.933333
4,NA1_5299791997,100,True,1,4,0,72553,65560,42,27,...,6983,0,16,29,9,13,0,8,75,36.466667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,NA1_5281683648,200,False,0,1,0,59279,56700,24,47,...,-12320,1,2,9,21,11,0,3,47,33.216667
334,NA1_5278373384,100,True,1,5,1,91198,80225,51,52,...,6751,2,17,40,30,42,0,11,142,40.866667
335,NA1_5278373384,200,False,1,2,0,84447,76933,52,51,...,-6751,1,7,33,35,8,0,11,95,40.866667
336,NA1_5278349815,100,True,1,3,0,79847,73658,41,42,...,6149,0,14,18,19,17,0,26,94,37.833333


In [20]:
silver_data["totalInhibitorsTaken"]

0      0
1      2
2      2
3      0
4      2
      ..
333    0
334    2
335    1
336    1
337    1
Name: totalInhibitorsTaken, Length: 338, dtype: int64